In [17]:
import torch
import numpy as np
import cv2
from time import time
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import datasets, transforms, models


In [6]:
labels_dict = {1: 'Eurasian_jay',
 2: 'great_spotted_woodpecker',
 3: 'greenfinch',
 4: 'blue_tit',
 5: 'Carduelis',
 6: 'common_redpoll',
 7: 'great_tit',
 8: 'bullfinch',
 9: 'Eurasian_siskin',
 10: 'Eurasian_tree_sparrow',
 11: 'hawfinch',
 12: 'willow_tit',
 13: 'Fieldfare',
 14: 'Common chaffinch'}

In [ ]:
'''
#load Faster R-CNN without weights
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
#load weights
model.load_state_dict(torch.load('/home/costia/birds/model_g2400_6_ep.pt'))

model = model.to(device)
'''

In [ ]:
#TODO Russian, English an japanese version of birds Name

In [12]:
class BirdsDetection:
    """
    Class implements detection with Faster R-CNN trained for 14 birds species
    """

    def __init__(self, source, out_file="detection_Video.avi",
                       weights_path = '/home/costia/birds/model_g2400_6_ep.pt',
                       conf_lvl = 0.6,labels_dict = labels_dict):
        """
        
        """
        self.source = source
        self.weights_path = weights_path
        self.model = self.load_model()
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.conf_lvl = conf_lvl
        self.transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                        ])


    def get_video_from_source(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :returns: opencv2 video capture object
        """
        return cv2.VideoCapture(self.source)

    def load_model(self):
        """
        load Faster R-CNN without weights pytorch hub and  
        load trained weights.
               
        :returns: Pytorch model with weights.
        """
        #
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        num_classes = 14+1 #(n_classes + background)
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
       #load weights
        
        model.load_state_dict(torch.load(self.weights_path))
        model.eval()
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame .
        :param frame: input frame in numpy/list/tuple format.
        :returns: Labels and  boxes of objects detected by model in the frame.
        """
        self.model.to(self.device)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        inputs = self.transform(frame)
        
        inputs = (inputs) #for img in frame]
        #plt.imshow(inputs)
        inputs.unsqueeze_(0)
        #print(inputs.shape)
        inputs = inputs.to(self.device) #for img in inputs]
        with torch.no_grad():
            preds = self.model(inputs)
        torch.cuda.empty_cache()
        
        return preds

    def class_to_label(self, label):
        """
        For a given label value, return bird species name.
        :param x: numeric label
        :return: corresponding bird species
        """
        return labels_dict [label]

    def plot_boxes(self, preds, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :returns: Frame with bounding boxes and labels ploted on it.
        """
        
        
        mask = preds[0]['scores'] >= self.conf_lvl
        print('Founded:',preds[0]['scores'])
        confidences = preds[0]['scores'][mask]
        boxes = preds[0]['boxes'][mask]
        labels = preds[0]['labels'][mask]
        
        flag = True
        
        for i, box in enumerate(boxes):
            x_min, y_min, x_max, y_max = map(int,box)
            bgr = (0, 255, 0)
            #print((x_min, y_min), (x_max, y_max))
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), bgr, 2)
            cv2.putText(frame, self.class_to_label(int(labels[i])), (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
        return frame
      

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :returns: void
        """
        player = self.get_video_from_source()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 30, (x_shape, y_shape))
        i = 0
        while True:
            
            start_time = time()
            ret, frame = player.read()
            assert ret
            if i%1==0:
                results = self.score_frame(frame)
                frame = self.plot_boxes(results, frame)
                end_time = time()
                fps = 1/np.round(end_time - start_time, 3)
                print(f"Frames Per Second : {fps}")
            
            i+=1
            out.write(frame)



In [16]:
# Create a new birds detection object and execute.
detect = BirdsDetection("/home/costia/VID-20211031-WA0001.mp4",conf_lvl=0.25,
                        weights_path = '/home/costia/birds/best_model.pt',
                        out_file='detected_birds__.avi')
detect()

Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 3.7593984962406015
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.1095, 0.1077, 0.0959], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.1103, 0.1063, 0.1028], device='cuda:0')
Frames Per Second : 4.926108374384236
Founded: tensor([0.1114, 0.1083, 0.0997], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.1085, 0.1003, 0.0910], device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([0.0997, 0.0940, 0.0814], device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([0.1069, 

Frames Per Second : 5.154639175257731
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([], device='cuda:0')
Frames Per Second : 4.8076923076923075
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([0.0504], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.0502], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([0.0506], device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([], device='cuda:0')
Fr

Founded: tensor([0.5113, 0.2961, 0.1897, 0.1189, 0.0869, 0.0545], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.3322, 0.2342, 0.1819, 0.0763, 0.0635, 0.0556], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.3962, 0.3083, 0.3047, 0.2526, 0.1359, 0.0856, 0.0813, 0.0623],
       device='cuda:0')
Frames Per Second : 5.0
Founded: tensor([0.3872, 0.2061, 0.1546, 0.0591], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([0.4144, 0.2483, 0.0698, 0.0601, 0.0520], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.2972, 0.2723, 0.0982, 0.0563, 0.0539, 0.0509], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.5828, 0.2677, 0.1301, 0.1167, 0.0930, 0.0906], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.6067, 0.2359, 0.2066, 0.1569, 0.1268, 0.1101, 0.0514],
       device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([0.6029, 0.2211

Founded: tensor([0.1607, 0.1343, 0.1091, 0.0690, 0.0629, 0.0604, 0.0515, 0.0511],
       device='cuda:0')
Frames Per Second : 4.9504950495049505
Founded: tensor([0.1166, 0.1045, 0.0905, 0.0689, 0.0510], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.1173, 0.1100, 0.0829, 0.0688, 0.0548, 0.0510], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.1267, 0.1020, 0.0896, 0.0689, 0.0521, 0.0511, 0.0504],
       device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([0.0689, 0.0671, 0.0559, 0.0510, 0.0508], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.2175, 0.1185, 0.0769, 0.0678, 0.0666, 0.0509], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([0.2638, 0.1058, 0.0682, 0.0679, 0.0615, 0.0510], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([0.2283, 0.0883, 0.0683, 0.0561, 0.0512], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor(

Frames Per Second : 5.208333333333333
Founded: tensor([0.4406, 0.2707, 0.2002, 0.1918, 0.1191, 0.1141, 0.1117, 0.1103, 0.0827,
        0.0812, 0.0791, 0.0704, 0.0623, 0.0548, 0.0525, 0.0502],
       device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([0.4038, 0.2700, 0.1817, 0.1280, 0.1251, 0.1187, 0.1143, 0.0936, 0.0874,
        0.0859, 0.0757, 0.0547, 0.0528], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.4529, 0.2706, 0.1634, 0.1451, 0.1160, 0.1011, 0.0824, 0.0789, 0.0627,
        0.0536, 0.0506], device='cuda:0')
Frames Per Second : 4.878048780487805
Founded: tensor([0.5938, 0.1610, 0.1602, 0.1550, 0.0864, 0.0828, 0.0772, 0.0691, 0.0639,
        0.0527, 0.0501], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.8605, 0.1220, 0.1062, 0.1026, 0.0932, 0.0650, 0.0587, 0.0535],
       device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.9134, 0.1097, 0.0781, 0.0768, 0.0732, 0.0556], device='cuda

...

Founded: tensor([0.0930, 0.0588, 0.0541], device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([0.0966, 0.0642, 0.0592], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.0963, 0.0673, 0.0618], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([0.1049, 0.0743, 0.0693], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.1010, 0.0704, 0.0700], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([0.0513], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.291005291005291
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.

Founded: tensor([], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.2356020942408374
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.319148936170213
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([], device='cuda:0')
Frames Per Second : 4.975124378109452
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([], device='cuda:0')
Frames Per Second : 4.975124378109452
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.2356020942408374
Founded: tensor([], 

Founded: tensor([], device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.025125628140703
Founded: tensor([], device='cuda:0')
Frames Per Second : 4.926108374384236
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.2356020942408374
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.2356020942408374
Founded: tensor([], device='cuda:0')
Frames Per Second : 4.9504950495049505
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([], de

Founded: tensor([0.1264, 0.0550], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([0.0549], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([0.0536], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.0577, 0.0507], device='cuda:0')
Frames Per Second : 5.208333333333333
Founded: tensor([0.0596, 0.0529], device='cuda:0')
Frames Per Second : 4.975124378109452
Founded: tensor([0.0582], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.0564], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.1020408163265305
Founded: tensor([], device='cuda:0')
Frames Per Second : 5.128205128205128
Founded: tensor([0.0960, 0.0650], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([0.0813, 0.0714], device='cuda:0')
Frames Per Second : 5.181347150259067
Founded: tensor([0.1024, 0.0698], device='cuda:0')
Frames Per Second : 5.07614213

AssertionError: 